In [1]:
%load_ext cython

## argsort

In [2]:
%%cython

# distutils: language = c++
# cython: cdivision = True
# cython: boundscheck = False
# cython: wraparound = False
# cython: profile = False


# https://github.com/jcrudy/cython-argsort/blob/master/cyargsort/test_argsort.py
cimport numpy as cnp
import numpy as np
from libc.stdlib cimport malloc, free
from cython cimport view
cnp.import_array() # need this line to access the numpy C-API
 
ctypedef cnp.float64_t FLOAT_t
ctypedef cnp.intp_t INT_t
ctypedef cnp.ulong_t INDEX_t
ctypedef cnp.uint8_t BOOL_t

cdef extern from "stdlib.h":
    ctypedef void const_void "const void"
    void qsort(void *base, int nmemb, int size,
            int(*compar)(const_void *, const_void *)) nogil

cdef struct Sorter:
    INT_t index
    FLOAT_t value

cdef int _compare(const_void *a, const_void *b):
    # cdef FLOAT_t v = ((<Sorter*>a)).value-((<Sorter*>b)).value
    cdef FLOAT_t v = (<Sorter*>a).value - (<Sorter*>b).value
    if v < 0:
        return -1
    else: 
        return 1

cdef void cyargsort(FLOAT_t[:] data, Sorter * order):
    cdef INT_t i
    cdef INT_t n = data.shape[0]
    for i in range(n):
        order[i].index = i
        order[i].value = data[i]
    qsort(<void *> order, n, sizeof(Sorter), _compare)
    
cpdef argsort(FLOAT_t[:] data, INT_t[:] order):
    cdef INT_t i
    cdef INT_t n = data.shape[0]
    cdef Sorter *order_struct = <Sorter *> malloc(n * sizeof(Sorter))
    cyargsort(data, order_struct)
    for i in range(n):
        order[i] = order_struct[i].index
    free(order_struct)

    
import numpy as np
import time
n = 10
data = np.random.normal(size=n)
order = np.empty(shape=n,dtype=np.intp)

time0 = time.time()
argsort(data, order)
time1 = time.time()
cython_time = time1 - time0

time0 = time.time()
np_order = np.argsort(data)
time1 = time.time()
np_time = time1 - time0

print('Cython time: %f' % cython_time)
print('Numpy time: %f' % np_time)

assert np.all(order == np.argsort(data))

Cython time: 0.000050
Numpy time: 0.000073


In [13]:
cimport cython
import numpy as np
from libc.string cimport memcpy
from libc.stdlib cimport malloc, free
from cython.parallel import parallel, prange
from scipy.linalg cimport cython_blas

cdef inline void outer(char *transa, char *transb, int *m, double* x, int* incx,
                       double* y, int* incy, double* A, int* lda) nogil:
    """
    transa: (input)
        On entry, TRANSA specifies the form of op( A ) to
        be used in the matrix multiplication as follows:

        TRANSA = 'N' or 'n',  op( A ) = A.
        TRANSA = 'T' or 't',  op( A ) = A'.

        Unchanged on exit.

    transb: (input)
        same as transa, just for op( B )

    m : (input)
        On entry,  M  specifies  the number  of  rows   of
               the   matrix  op(  A  )  and of the  matrix  C.  M
               must  be at least  zero.  Unchanged on exit.
    n : (input)
               On entry,  N  specifies the number  of columns  of
               the  matrix  op(  B ) and the number of columns of
               the matrix C. N must be at least zero.   Unchanged
               on exit.

     K (input)
        On entry,  K  specifies  the number of columns  of
               the  matrix  op( A ) and the number of rows of the
               matrix  op(  B  ).  K  must  be  at  least   zero.
               Unchanged on exit.

     ALPHA (input)
               On  entry,  ALPHA  specifies  the  scalar   alpha.
               Unchanged on exit.

     A (input)
               DOUBLE PRECISION array of DIMENSION ( LDA,  ka  ),
               where  ka is k  when  TRANSA = 'N' or 'n',  and is
               m  otherwise.  Before entry with  TRANSA = 'N'  or
               'n',   the  leading   m  by k part of the array  A
               must contain the matrix  A,  otherwise the leading
               k  by  m   part of the array  A  must contain  the
               matrix A.  Unchanged on exit.

     LDA (input)
               On entry, LDA specifies the first dimension  of  A
               as  declared  in  the  calling (sub) program. When
               TRANSA = 'N' or 'n' then LDA >= max( 1, m ),  oth-
               erwise  LDA >= max( 1, k ).  Unchanged on exit.

     B (input)
               DOUBLE PRECISION array of DIMENSION ( LDB,  kb  ),
               where  kb is n  when  TRANSB = 'N' or 'n',  and is
               k  otherwise.  Before entry with  TRANSB = 'N'  or
               'n',   the  leading   k  by n part of the array  B
               must contain the matrix  B,  otherwise the leading
               n  by  k   part of the array  B  must contain  the
               matrix B.  Unchanged on exit.

     LDB (input)
               On entry, LDB specifies the first dimension  of  B
               as  declared  in  the  calling (sub) program. When
               TRANSB = 'N' or 'n' then LDB >= max( 1, k ),  oth-
               erwise LDB >= max( 1, n ).  Unchanged on exit.
     BETA (input)
               On entry,  BETA  specifies the scalar  beta.  When
               BETA   is  supplied as zero then C need not be set
               on input.  Unchanged on exit.

     C (input/output)
               DOUBLE PRECISION array of DIMENSION (  LDC,  n  ).
               Before  entry,  the  leading   m by n  part of the
               array  C must contain the matrix  C,  except  when
               beta   is zero, in which case C need not be set on
               entry.  On exit, the array  C  is  overwritten  by
               the   m  by  n   matrix  ( alpha*op( A )*op( B ) +
               beta*C ).

     LDC (input)
               On entry, LDC specifies the first dimension  of  C
               as declared in  the  calling  (sub)  program.  LDC
               >= max( 1, m ).  Unchanged on exit.
 
    """
    cython_blas.dgemm(m, n, alpha, x, incx, y, incy, A, lda)
    
x = np.array([[9, 4], [3, 1], [5, 2]], dtype = np.float64)

a = np.array([[ 5, 1 ,3], 
                  [ 1, 1 ,1], 
                  [ 1, 2 ,1]])
b = np.array([1, 2, 3])
print(a.dot(b))

cdef:
    double[:] B = b
    double[:, :] A = a
    
    

# cdef:
#     int i, one = 1
#     double alpha = 1.0
#     double *A
#     double[:, :] X = x
#     int n_factors = X.shape[1]
#     double[:, :] end_result = np.zeros((n_factors, n_factors), dtype = np.float64)


# A = <double *> malloc(sizeof(double) * n_factors * n_factors)
# memcpy(A, &end_result[0, 0], sizeof(double) * n_factors * n_factors)
# memcpy(&end_result[0, 0], A, sizeof(double) * n_factors * n_factors)
# print(np.asarray(end_result))

# for i in range(X.shape[0]):
#     print(np.asarray(X[i]))
#     outer(&n_factors, &n_factors, &alpha, &X[i, 0], &one, &X[i, 0], &one, A, &n_factors)
#     memcpy(&end_result[0, 0], A, sizeof(double) * n_factors * n_factors)
#     print(np.asarray(end_result))

free(A)
print(x.T.dot(x))

Cython time: 0.000504
Numpy time: 0.000036
[16  6  8]
[[ 0.  0.]
 [ 0.  0.]]
[ 9.  4.]
[[ 81.  36.]
 [ 36.  16.]]
[ 3.  1.]
[[ 90.  39.]
 [ 39.  17.]]
[ 5.  2.]
[[ 115.   49.]
 [  49.   21.]]
[[ 115.   49.]
 [  49.   21.]]


In [10]:
import numpy as np
import time
n = 10
data = np.random.normal(size=n)
order = np.empty(shape=n,dtype=np.intp)

time0 = time.time()
argsort(data, order)
time1 = time.time()
cython_time = time1 - time0

time0 = time.time()
np_order = np.argsort(data)
time1 = time.time()
np_time = time1 - time0

print('Cython time: %f' % cython_time)
print('Numpy time: %f' % np_time)

assert np.all(order == np.argsort(data))

Cython time: 0.000054
Numpy time: 0.000062


In [4]:
def compute_apk(y_true, y_pred, k):
    """
    average precision at k, y_pred is assumed 
    to be truncated to length k prior to feeding
    it to the function
    """
    # convert to set since membership 
    # testing in a set is vastly faster
    actual = set(y_true)
    
    # precision at i is a percentage of correct 
    # items among first i recommendations; the
    # correct count will be summed up by n_hit
    n_hit = 0
    precision = 0
    for i, p in enumerate(y_pred, 1):
        if p in actual:
            n_hit += 1
            precision += n_hit / i

    # divide by recall at the very end
    avg_precision = precision / min(len(actual), k)
    print(precision)
    return avg_precision

In [5]:
import numpy as np
k = 2
y_true = np.array([1, 2, 3, 4, 5])
y_pred = np.array([6, 4, 7, 1, 2])
compute_apk(y_true, y_pred[:k], k) # 0.25

0.5


0.25